# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [13]:
from typing import Tuple, List
from math import inf

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
import random as rd

Wielkość planszy

In [14]:
ROW_COUNT = 6
COLUMN_COUNT = 7
CURRENT_PLAYER = None
DEPTH = 1

In [15]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game

    def evaluate_position(self, player: Player)->float:
        score = 0
        board = self.game.state.fields

        # pionowo
        for column in range(COLUMN_COUNT):
            temp_column = [board[column][i] for i in range(ROW_COUNT)]
            for row in range(3):
                check_area = temp_column[row:row+4]
                score += self.evaluate_area(check_area, player)

        # poziomo
        for row in range(ROW_COUNT):
            temp_row = [board[i][row] for i in range(COLUMN_COUNT)]
            for column in range(3):
                check_area = temp_row[column:column+4]
                score += self.evaluate_area(check_area, player)
        
        # po skosie
        for row in range(ROW_COUNT-3):
            for column in range(COLUMN_COUNT-3):
                check_area = [board[column+i][row+i] for i in range(4)]
                score += 0.7 * self.evaluate_area(check_area, player)
                check_area = [board[column+3-i][row+i] for i in range(4)]
                score += 0.7 * self.evaluate_area(check_area, player)

        return score
    
    def evaluate_area(self, check_area, player):
        score = 0
        if player.char == self.game.first_player.char:
            main_player = self.game.first_player
            other_player = self.game.second_player
        else:
            main_player = self.game.second_player
            other_player = self.game.first_player
        
        if check_area.count(main_player) == 4:
            score += 100
        elif check_area.count(main_player) == 3 and check_area.count(None) == 1:
            score += 13
        elif check_area.count(main_player) == 2 and check_area.count(None) == 2:
            score += 3
        elif check_area.count(main_player) == 1 and check_area.count(None) == 3:
            score += 0.5
        if check_area.count(other_player) == 4:
            score -= 100
        elif check_area.count(other_player) == 3 and check_area.count(None) == 1:
            score -= 5
        elif check_area.count(other_player) == 2 and check_area.count(None) == 2:
            score -= 0.5
        return score

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT-1] == None
    
    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:    

        if depth == 0 or self.game.get_winner() is not None:
            if isinstance(self.game.get_winner(), Player):
                if self.game.get_winner().char == CURRENT_PLAYER.char:
                    return (None, 10000)
                elif self.game.get_winner().char != CURRENT_PLAYER.char:
                    return (None, -10000)
                else:
                    return (None, 0)
            else:
                return (None, self.evaluate_position(CURRENT_PLAYER))
        if is_maximizing_player:
            score = -inf
            for column in range(COLUMN_COUNT):
                if self.is_valid_move(column):
                    if depth  == DEPTH:
                        if CURRENT_PLAYER.char =='a':
                            temp_game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.first_player, second_player=self.game.second_player)
                        else:
                            temp_game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)
                    else:
                        temp_game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)
                    temp_game.state.fields = self.game.state.fields
                    temp_game.make_move(ConnectFourMove(column))
                    temp_solver = MinMaxSolver(temp_game)
                    temp_score = temp_solver.minimax(depth-1, alpha, beta, False)[1]

                    if temp_score > score:
                        score = temp_score
                        best_move = column

                    alpha = max(alpha, score)
                    if alpha >= beta:
                        break

        else:
            score = inf
            for column in range(COLUMN_COUNT):
                if self.is_valid_move(column):
                    temp_game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)
                    temp_game.state.fields = self.game.state.fields
                    temp_game.make_move(ConnectFourMove(column))
                    temp_solver = MinMaxSolver(temp_game)
                    temp_score = temp_solver.minimax(depth-1, alpha, beta, True)[1]

                    if temp_score < score:
                        score = temp_score
                        best_move = column

                    beta = min(beta, score) 
                    if alpha >= beta:
                        break

        return (best_move, score)

Rozgrywka; co zrobić z funkcją best_move

In [17]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

DEPTH = 5 # 1->fast win, 2->dużo blokowania, 3-> 3 z dwoma pustymi, 4->blok przed dwoma pustymi
alpha = -inf
beta = inf

while not game.is_finished():
    CURRENT_PLAYER = game.get_current_player()
    move = solver.minimax(DEPTH, alpha, beta, True)
    game.make_move(ConnectFourMove(move[0]))
    print(game)

winner = game.get_winner()
if winner is None:
    print('Draw!')
else:
    print('Winner: Player ' + winner.char)


Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][b][a][ ][

WNIOSKI: Jakość ruchów wykonywanych przez komputer zależy bezpośrednio od głębokości. Im głębsze przeszukiwanie tym algorytm działa dłużej i przewiduje lepsze ruchy. W przypadku algorytmu mini-max bez odcianania w każdej iteracji program sprawdza 7^{depth} co powoduje długie działanie programu. Dzięki odcinaniu alpha-beta jesteśmy w stanie znacznie zmniejszyć drzewo które przeszukujemy oszczędzając dużo czasu (dla głębokości 5: bez odcinania -> 2min.24s.; z odcinaniem -> 23s.). Poprawność działania mocno zależy od dobranej heurystyki i zmiana jej wpływa na dobór najlepszego ruchu.    

In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
# [b][ ][ ][a][ ][ ][ ]
# [a][ ][ ][a][ ][ ][ ]
# [b][ ][ ][b][ ][ ][ ]
# [a][ ][ ][a][ ][ ][ ]
# [b][a][a][b][ ][ ][ ]
# [a][b][a][b][b][b][ ]
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(0)) #b

# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(1))
# game.make_move(ConnectFourMove(1))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(2)) #a

# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(3)) #a
# game.make_move(ConnectFourMove(4))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(5))

# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][a][ ][ ][ ]
# [ ][ ][a][b][ ][ ][ ]
# [ ][ ][a][b][ ][ ][ ]

game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(2))

# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][b][ ][ ][ ][ ]
# [ ][ ][a][a][ ][ ][b]
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove())

# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][b][ ][ ][ ][ ]
# [ ][ ][a][a][a][b][b]

# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(5))
# game.make_move(ConnectFourMove(4))
# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(2))

# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][a][ ][ ][ ]
# [ ][ ][a][a][ ][ ][b]
# [b][b][a][a][ ][ ][b]

# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(1))
# game.make_move(ConnectFourMove(3))

print(game)
alpha = -inf
beta = inf
DEPTH = 3
solver = MinMaxSolver(game)
# score = solver.evaluate_position(game.get_current_player())
CURRENT_PLAYER = game.get_current_player()
move= solver.minimax(DEPTH, alpha, beta, True)
game.make_move(ConnectFourMove(move[0]))
print(game)


winner = game.get_winner()
if winner is None:
    print('Draw!')
else:
    print('Winner: Player ' + winner.char)


Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
Draw!


In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][ ][ ][ ][ ][ ]
# [ ][ ][a][a][ ][ ][b]
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(2))

alpha = -inf
beta = inf
DEPTH = 4
solver = MinMaxSolver(game)
CURRENT_PLAYER = game.get_current_player()
move= solver.minimax(DEPTH, alpha, beta, True)
game.make_move(ConnectFourMove(move[0]))
print(game)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][b]


In [ ]:
from time import sleep

p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

# print('Difficulty:')
# print('1. Easy')
# print('2. Midium')
# print('3. Hard')
# choice = input()
# if choice == '1':
#     DEPTH = 1
# elif choice == '2':
#     DEPTH = 3
# else:
#     DEPTH = 5
DEPTH = 5
alpha = -inf
beta = inf
i = 0
print(game)

while not game.is_finished():
    if i % 2 == 0:
        move = input()
        game.make_move(ConnectFourMove(int(move)))
    else:
        sleep(2)
        CURRENT_PLAYER = p2
        if i == 5:
            pass
        move = solver.minimax(DEPTH, alpha, beta, True)
        game.make_move(ConnectFourMove(move[0]))
    print(game)
    i += 1

winner = game.get_winner()
if winner is None:
    print('Draw!')
else:
    print('Winner: Player ' + winner.char)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]


ValueError: invalid literal for int() with base 10: ''